In [ ]:
# !pip uninstall pymatgen -y
# !pip uninstall numpy -y
# !pip install numpy==1.21.0
# !pip install "pymatgen<2021"

In [1]:
from moffragmentor import MOF

In [2]:
from rdkit.Chem import Draw

In [13]:
mof = MOF.from_cif("../tests/test_files/HKUST-1.cif")

In [14]:
fragments = mof.fragment()

[] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [15]:
linker = fragments.linkers[0].rdkit_mol

In [16]:
from rdkit.Chem import SanitizeMol

In [17]:
from rdkit.Chem.rdmolops import GetFormalCharge, AddHs

In [18]:
fragments.nodes[0].rdkit_mol

In [19]:
def count_radical_electrons(mol):
    radical_electrons = []
    for atom in mol.GetAtoms():
        radical_electrons.append(atom.GetNumRadicalElectrons())
    return sum(radical_electrons)

In [20]:
count_radical_electrons(fragments.nodes[0].rdkit_mol)

10

In [21]:
count_radical_electrons(fragments.linkers[2].rdkit_mol)

3

In [22]:
count_radical_electrons(fragments.linkers[3].rdkit_mol)

3

In [23]:
Draw.MolToFile(fragments.linkers[3].rdkit_mol, 'fragment_linker_3.png')

In [24]:
fragments.linkers[3].rdkit_mol

In [25]:
from oximachinerunner import OximachineRunner

In [26]:
runner = OximachineRunner()

In [27]:
runner.run_oximachine("ABEDAQ.cif")

/home/kjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:745: UserWarning: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
  warnings.warn(msg)
/home/kjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:1193: UserWarning: Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
  warnings.warn(


OrderedDict([('metal_indices', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
             ('metal_symbols',
              ['Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co',
               'Co']),
             ('prediction', [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
             ('max_probas',
              [0.9324728272919917,
               0.9324728272919917,
               0.9042644535892913,
               0.9324728272919917,
               0.9042644535892913,
               0.9042644535892913,
               0.9324728272919917,
               0.9324728272919917,
               0.9324728272919917,
               0.9324728272919917,
               0.9324728272919917,
               0.9042644535892913]),
             ('base_predictions',
              [[2, 2, 2, 2],
               [2, 2, 2, 2],
               [2, 2, 2, 2],
     

In [28]:
fragments.nodes[0].rdkit_mol

In [29]:
from moffragmentor.utils import visualize_part

In [30]:
def get_leftover_indices(s, ignore_indices, metal_indices):
    ignore_indices = [i for i in ignore_indices if i not in metal_indices] 
    return [i for i in range(len(s)) if i not in ignore_indices]

In [31]:
node = fragments.nodes[2]

In [32]:
node.binding_indices

[1, 2]

In [33]:
node.connecting_indices

[11, 5, 9, 7]

In [34]:
from pymatgen.core import Molecule

In [35]:
def create_molecule_without_sites(molecule, ignore_sites, metal_sites): 
    indices = get_leftover_indices(node.molecule, ignore_sites, metal_sites)

    sites = []

    for i, s in enumerate(molecule):
        if i in indices: 
            sites.append(s)

    return Molecule.from_sites(sites)


In [36]:
submodel = create_molecule_without_sites(
    node.molecule,
    node.binding_indices + node.connecting_indices + node.graph_branching_indices,
    node.metal_indices,
)


In [37]:
import nglview as nv

In [38]:
node.show_binding_structure()

NGLWidget()

In [39]:
nv.show_pymatgen(submodel)

NGLWidget()

In [40]:
node.show_binding_structure()

NGLWidget()

In [41]:
node.show_connecting_structure()

NGLWidget()

In [42]:
mof.show_structure()

NGLWidget()

In [43]:
node.show_molecule()

NGLWidget()

In [44]:
len(node.binding_indices)

2

In [45]:
node.original_binding_indices

[2, 36]

In [46]:
node.molecule[5]

Site: C (18.5191, 20.9954, 28.1607)